In [31]:
import random
import time
import numpy as np
import matplotlib.pyplot as plt
import pandas as pd
import altair as alt
import tqdm
from scipy import sparse
from sklearn.linear_model import LogisticRegression
from sklearn.ensemble import RandomForestRegressor
from functools import lru_cache
from modAL.models import ActiveLearner
from sklearn.decomposition import PCA, TruncatedSVD
from functools import partial
from modAL.batch import uncertainty_batch_sampling
from modAL.models import BayesianOptimizer
from modAL.acquisition import optimizer_EI, max_EI
from sklearn.metrics import recall_score
from xgboost import XGBRegressor

In [2]:
alt.renderers.enable('default')

RendererRegistry.enable('default')

In [3]:
NUM_CHUNKS = 1

# Generate some indices
Even the sparse matrices won't fit in memory. So we will have to loop through them when making predictions or sampling random items.

In [4]:
USE_EMBEDDINGS = True

In [5]:
RECEPTOR = "EnamineHTS"
DATA_DIR = "/mnt/efs/enamine"

if USE_EMBEDDINGS:
    OUTPUT_RESULTS_FILE = f"{RECEPTOR}_embedding_results.csv"
else:
    OUTPUT_RESULTS_FILE = f"{RECEPTOR}_results.csv"

In [6]:
# count number of items:
indptr = [0]
scores_lst = []

for chunk_id in range(NUM_CHUNKS):
    scores = np.load(f"{DATA_DIR}/{RECEPTOR}_scores_{chunk_id}.npy")
    indptr.append(indptr[-1] + scores.shape[0])
    scores_lst.append(scores)
    
scores = np.concatenate(scores_lst)

In [7]:
scores.shape

(2104318,)

In [9]:
def load_vectors(chunk_id, use_embeddings=True):
    print("Loading vectors", end="; ", flush=True)
    if use_embeddings:
        vectors = np.load(f"{DATA_DIR}/{RECEPTOR}_embeddings_{chunk_id}.npy")
    else:
        vectors = sparse.load_npz(f"{DATA_DIR}/{RECEPTOR}_fingerprints_{chunk_id}.npz")
    return vectors

def extract_vectors(chunk_id, indptr, is_train):
    print(f"Extracting vectors: {chunk_id}", end="; ", flush=True)
    vectors = load_vectors(chunk_id, use_embeddings=USE_EMBEDDINGS)
    mask = is_train[indptr[chunk_id]:indptr[chunk_id+1]]
    return vectors[mask].astype(int)

def build_train(indptr, is_train):
    print("Building training set", end="; ", flush=True)
    if USE_EMBEDDINGS:
        vectors = np.vstack([extract_vectors(i, tuple(indptr), is_train) for i in range(NUM_CHUNKS)])
    else:
        vectors = sparse.vstack([extract_vectors(i, tuple(indptr), is_train) for i in range(NUM_CHUNKS)])  
    return vectors

def chunk_predict_proba(model, indptr, is_train):
    print("Predicting proba", end="; ", flush=True)
    probas = []
    for chunk_id in range(NUM_CHUNKS):
        vectors = extract_vectors(chunk_id, indptr, ~is_train)
        proba = model.predict_proba(vectors)[:,1]
        probas.append(proba)
    return np.concatenate(probas)

def chunk_predict(model, indptr, is_train):
    print("Predicting scores", end="; ", flush=True)
    preds = []
    for chunk_id in range(NUM_CHUNKS):
        vectors = extract_vectors(chunk_id, indptr, ~is_train)
        pred = model.predict(vectors)
        preds.append(pred)
    return np.concatenate(preds)

# Train a Logistic Regression models

In [11]:
# model = LogisticRegression(max_iter=10000, C=1)

In [12]:
# model = XGBRegressor(
# #     objective="reg:squaredlogerror"
#     use_label_encoder=False
# )

In [37]:
def greedy(Y_mean: np.ndarray) -> np.ndarray:
    """Greedy acquisition score
    
    Parameters
    ----------
    Y_mean : np.ndarray
        the mean predicted y values
    Returns
    -------
    np.ndarray
        the greedy acquisition scores
    """
    return Y_mean

def ucb(Y_mean: np.ndarray, Y_var: np.ndarray, beta: int = 2) -> float:
    """Upper confidence bound acquisition score
    Parameters
    ----------
    Y_mean : np.ndarray
    Y_var : np.ndarray
        the variance of the mean predicted y values
    beta : int (Default = 2)
        the number of standard deviations to add to Y_mean
    Returns
    -------
    np.ndarray
        the upper confidence bound acquisition scores
    """
    return Y_mean + beta*np.sqrt(Y_var)

In [38]:
def get_means_and_vars(model, x):
    preds = np.zeros((len(x), len(model.estimators_)))

    for j, submodel in enumerate(model.estimators_):
        preds[:, j] = submodel.predict(x)

    return np.mean(preds, axis=1), np.var(preds, axis=1)

In [39]:
model = RandomForestRegressor(
    n_estimators=100,
    max_depth=8,
    min_samples_leaf=1
)

In [40]:
x_raw = load_vectors(chunk_id=0, use_embeddings=True)

Loading vectors; 

In [41]:
TOP_K_THRESHOLD = 1_000
N_QUERIES = 5
N_FOLDS = 3
FRACTION = 0.004
# PERCENTILE = 0.3
BATCH_SIZE = int(len(scores)*FRACTION)
preset_batch = partial(uncertainty_batch_sampling, n_instances=BATCH_SIZE)

In [42]:
n_labeled_examples = scores.shape[0]
train_indices = np.array(random.sample(range(n_labeled_examples+1), k=BATCH_SIZE))

In [43]:
mask = np.zeros(scores.shape[0]).astype(bool)
mask[train_indices] = True

In [44]:
# cutoff = np.percentile(scores[train_indices], PERCENTILE)

In [45]:
x_test = x_raw
y_test = (scores.argsort().argsort() < TOP_K_THRESHOLD)
y_raw = scores
# y_raw = scores < cutoff

In [46]:
x_train = x_raw[mask]
y_train = y_raw[mask]
x_pool = x_raw[~mask]
y_pool = y_raw[~mask]

In [28]:
results = []

instances_seen = BATCH_SIZE

y_pred = learner.predict(X=x_test)
recall = recall_score(y_true=y_test, y_pred=y_pred)

print(f"Iteration: -1, Recall: {recall}")
results.append({
    "Training size": BATCH_SIZE, 
    "N ligands explore": instances_seen,
    "% top-k found": recall
})

start = time.time()

for index in range(N_QUERIES-1):
    fn_start = time.time()
    print("Querying instances...")
    query_index, query_instance = learner.query(x_pool)
    print(f"Took {(time.time() - fn_start)/60} minutes")
    
    fn_start = time.time()
    print("Teaching...")
    # Teach our ActiveLearner model the record it has requested.
    x, y = x_pool[query_index], y_pool[query_index]
    learner.teach(X=x, y=y)
    print(f"Took {(time.time() - fn_start)/60} minutes")
    
    # Remove the queried instance from the unlabeled pool.
    pool_mask = np.zeros(x_pool.shape[0]).astype(bool)
    pool_mask[query_index] = True
    x_pool = x_pool[~pool_mask]
    y_pool = y_pool[~pool_mask]
    
    fn_start = time.time()
    print("Predicting...")
    y_pred = learner.predict(X=x_test)
    recall = recall_score(y_true=y_test, y_pred=y_pred)
    instances_seen += BATCH_SIZE
    print(f"Took {(time.time() - fn_start)/60} minutes")
    
    print(f"Iteration: {index}, Recall: {recall}")
    
    results.append({
        "Training size": BATCH_SIZE, 
        "N ligands explore": instances_seen,
        "% top-k found": recall
    })
    
print(f"Took {(time.time() - start)/60} minutes")

ValueError: Classification metrics can't handle a mix of binary and continuous targets

In [ ]:
# training_set_fractions = [0.004, 0.002, 0.001]
training_set_fractions = [0.004]

percentile = 0.3

df = pd.DataFrame(columns=['Algorithm', 'Training size', 'N ligands explored', '% top-k found'])
count = 0

for i in range(3):
    idx = np.arange()
    np.random.shuffle(idx)

    for fraction in training_set_fractions:
        size = int(len(scores) * fraction)
        
        # split indices into train and test:
        train_indices = idx[:size].copy()
        test_indices = idx[size:].copy()
        train_indices.sort()
        test_indices.sort()

        # generate a 'is a training instance' mask. 
        is_train = np.zeros(scores.shape[0]).astype(bool)
        is_train[train_indices] = True

        # top_k molecules already found in the training set:
        num_found = top_k[train_indices].sum()

        df.loc[count] = ["morgan_feat", size, train_indices.shape[0], num_found/total]
        count += 1
        print(f"Iteration: {count}, Found {num_found} top k ligands")

        # estimate the cutoff once, from the initial random sample:
        cutoff = np.percentile(scores[train_indices], percentile)

        for i in range(5):
            # fit logreg model:
            x_train = build_train(indptr, is_train)
            y_train = scores[is_train] < cutoff

            print("Fitting model", end="; ", flush=True)
            model.fit(x_train, y_train)

            # predict (slowest step) for logreg:
            proba = chunk_predict_proba(model, indptr, is_train)

            # rank the probabilities
            proba_sorted = (-proba).argsort()

            # rank the unseen instances:
            test_indices = test_indices[proba_sorted]

            # now append the next N instances from the rank ordered unseen instances onto the training set:
            train_indices = np.concatenate([train_indices, test_indices[:size]])

            # update the isTrain mask and remove those training instances from the test set
            is_train[train_indices] = True
            test_indices = test_indices[size:]

            # keep the train and test idx arrays sorted so they agree with the chunked* methods:
            test_indices.sort()
            train_indices.sort()

            # topK molecules already found in the training set:
            num_found = top_k[train_indices].sum()

            df.loc[count] = ['morgan_feat', size, train_indices.shape[0], num_found/total]
            count += 1
            
            print(f"\nIteration: {count}, Found {num_found} top k ligands")
            
            df.to_csv(f"{DATA_DIR}/{OUTPUT_RESULTS_FILE}")

df.to_csv(f"{DATA_DIR}/{OUTPUT_RESULTS_FILE}")

# Results look like this:
And they can be plotted using `./plot_scripts/plot_wholedataset.py`

In [ ]:
df1 = pd.read_csv(f"{DATA_DIR}/{RECEPTOR}_embedding_results.csv", index_col=0)
df1['Algorithm'] = 'LogReg (embeddings)'

In [ ]:
# df2 = pd.read_csv(f"{DATA_DIR}/{RECEPTOR}_results.csv", index_col=0)
# df2['Algorithm'] = 'LogReg (fps)'

In [ ]:
df = pd.concat([df1])

In [ ]:
prev_results = [['RF (Graff)', 8_417, 84.3, 1.1], 
                ['NN (Graff)', 8_417, 95.7, 0.1],
                ['MPN (Graff)',8_417, 97.6, 0.3],
                ['random', 8_417, 2.6, 0.1],
                ['RF (Graff)', 4_208, 72.3, 1.9],
                ['NN (Graff)', 4_208, 88.8, 0.8],
                ['MPN (Graff)', 4_208, 93.3, 0.9],
                ['random', 4_208, 1.3, 0.4],
                ['RF (Graff)', 2_104, 55.8, 4.9],
                ['NN (Graff)', 2_104 , 70.5, 1.8],
                ['MPN (Graff)', 2_104, 78.5, 2.2],
                ['random', 2_104, 0.6, 0.2]]

coley = pd.DataFrame(columns=['Algorithm', 'Training size', 'N ligands explored', '% top-k found'])
count = 0 
for res in prev_results:
    desired_std_dev = res[3]
    samples = np.array([-1,0,1]).astype(float)
    samples *= (desired_std_dev/np.std(samples))
    for s in samples:
        coley.loc[count]= [res[0], res[1], res[1]*6, (s+res[2])/100]
        count += 1

In [ ]:
concat = pd.concat([df, coley])
concat['% top-k found'] *= 100
concat.columns = ['Algorithm', 'Training set size', 'N ligands explored', '% top-k found']
concat['Training set size'] = concat['Training set size'].apply(lambda num: f"{num:,d}",)

In [ ]:
error_bars = alt.Chart(concat).mark_errorbar(extent='ci').encode(
  x=alt.X('N ligands explored:Q',title='Number of ligands sampled'),
  y=alt.Y('% top-k found:Q', title='% top 1,000 found'),
    color=alt.Color('Algorithm')
)

points = alt.Chart(concat).mark_point(filled=False, size=40, color='black').encode(
  x=alt.X('N ligands explored:Q'),
  y=alt.Y('% top-k found:Q',aggregate='mean',title='% top 1,000 found'),
    color=alt.Color('Algorithm'),
    tooltip=alt.Tooltip('% top-k found:Q',aggregate='mean',title='% top 1,000 found')
)

line = alt.Chart(concat).mark_line(color='black',size=2,opacity=0.5).encode(
  x=alt.X('N ligands explored:Q'),
  y=alt.Y('% top-k found:Q',aggregate='mean',title='% top 1,000 found'),
    color=alt.Color('Algorithm')
)

ch = (error_bars+points+line).properties(height=300,width=150).facet(
    column=alt.Column('Training set size:N',sort=alt.Sort([0.004, 0.002, 0.001])),
).resolve_scale(x='independent')
# ch.save('../../figures/active_learning_percentage.html')

In [ ]:
ch

# PCA

In [ ]:
vectors = load_vectors(chunk_id=0, use_embeddings=False)

In [ ]:
classes = scores < cutoff

In [ ]:
# pca = PCA(n_components=2, random_state=42)
pca = TruncatedSVD(n_components=2, random_state=42)
transformed_vectors = pca.fit_transform(X=vectors)

# Isolate the data we'll need for plotting.
x_component, y_component = transformed_vectors[:, 0], transformed_vectors[:, 1]

In [ ]:
def plot_pca(x_component, y_component, classes):
    # Plot our dimensionality-reduced (via PCA) dataset.
    plt.figure(figsize=(8.5, 6), dpi=130)
    plt.scatter(x=x_component, y=y_component, c=classes, s=5, alpha=0.5)
    plt.title('Ligands after PCA transformation')
    plt.show()

In [ ]:
plot_pca(x_component=x_component, y_component=y_component, classes=classes)